In [0]:
#Importación de librerías
try:
  
  #Para usar colab
  from google.colab import drive
  drive.mount('/content/drive')
  
  import numpy as np
  import pandas as pd
  
except:
  
  import numpy as np
  import pandas as pd

import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline

import seaborn as sns
sns.set()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Lectura de datos
try:
  #Caso en que se este usando jupyter notebooks
  data_pulsar = pd.read_csv('HTRU_2_con_header.csv')
except:
  #Caso en que se use colab, cambiar directorio de acuerdo al drive
  dir_str ='drive/My Drive/CURSOS - MAESTRIA/APRENDIZAJE AUTOMÁTICO/MACHINE LEARNING - PROYECTO/ML_PULSAR_CLASSIFICATION_PROJECT/PREPROCESAMIENTO DE DATA/HTRU_2_sin_header.csv'
  #Se definen  los headers
  head = ['mean_IP', 'std_IP','kurt_IP','skew_IP','mean_DM', 'std_DM','kurt_DM','skew_DM','is_pulsar']
  #Lectura de la data
  data_pulsar = pd.read_csv(dir_str,header=None, names=head)

#Creacion del Fuzzy-KNN
El algoritmo a describir fui utilizado en el articulo  ["Pulsar Selection Using Fuzzy knn Classifier"](hhttps://www.researchgate.net/publication/321987873_Pulsar_Selection_Using_Fuzzy_knn_Classifier) del autor Taha Mohamed en el 2017, para la implementación se usoel código desarrollado en [link](https://github.com/sahilsehwag/FuzzyKNN/blob/master/fuzzy_knn.ipynb) 

In [0]:
#El codigo se tomo de la siguiente página: https://github.com/sahilsehwag/FuzzyKNN/blob/master/fuzzy_knn.ipynb
import operator
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score, confusion_matrix

class FuzzyKNN(BaseEstimator, ClassifierMixin):
    def __init__(self, k=3, plot=False, m=0.3):
        self.k = k
        self.plot = plot
        self.m = m   #Se agrego esta linea para controlar el valor de m, el cual por defecto estamos colocando 0.3 evaluado en el paper
        
        
    def fit(self, X, y=None):
        self._check_params(X,y)
        self.X = X
        self.y = y
        
        self.xdim = len(self.X[0])
        #print("xdim: ",self.xdim)
        self.n = len(y)
        #print("n: ",self.n)
        
        classes = list(set(y))
        classes.sort()
        self.classes = classes
        #print("classes: ",self.classes)
        
        self.df = pd.DataFrame(self.X)
        self.df['y'] = self.y
        
        self.memberships = self._compute_memberships()
        
        self.df['membership'] = self.memberships
        
        self.fitted_ = True
        return self
    
    
    def predict(self, X):
        if self.fitted_ == None:
            raise Exception('predict() called before fit()')
        else:
            m = self.m
            y_pred = []
            
            for x in X:
                neighbors = self._find_k_nearest_neighbors(pd.DataFrame.copy(self.df), x)
                
                votes = {}
                for c in self.classes:
                    den = 0
                    for n in range(self.k):
                        dist = np.linalg.norm(x - neighbors.iloc[n,0:self.xdim])
                        den += 1 / (dist ** (2 / (m-1)))
                    
                    neighbors_votes = []
                    for n in range(self.k):
                        dist = np.linalg.norm(x - neighbors.iloc[n,0:self.xdim])
                        num = (neighbors.iloc[n].membership[c]) / (dist ** (2 / (m-1)))
                        
                        vote = num/den
                        neighbors_votes.append(vote)
                    votes[c] = np.sum(neighbors_votes)
                    
                pred = max(votes.items(), key=operator.itemgetter(1))[0]
                y_pred.append(pred)
                #y_pred.append((pred, votes))
                
            return y_pred    
        
    def score(self, X, y):
        if self.fitted_ == None:
            raise Exception('score() called before fit()')
        else:
            predictions = self.predict(X)
            y_pred = [t[0] for t in predictions]
            confidences = [t[1] for t in predictions]
            
            return accuracy_score(y_pred=y_pred, y_true=y)
    
        
    def _find_k_nearest_neighbors(self, df, x):
        X = df.iloc[:,0:self.xdim].values
        
        df['distances'] = [np.linalg.norm(X[i] - x) for i in range(self.n)]
        
        df.sort_values(by='distances', ascending=True, inplace=True)
        neighbors = df.iloc[0:self.k]
        
        return neighbors

                
    def _get_counts(self, neighbors):
        groups = neighbors.groupby('y')
        counts = {group[1]['y'].iloc[0]:group[1].count()[0] for group in groups}
        
        return counts
        
        
    def _compute_memberships(self):
        memberships = []
        for i in range(self.n):
            #esta lineas se insertaron para probar el numero de iteración del algoritmo
            #if i%10000== 0:
              #print(i)
            x = self.X[i]
            y = self.y[i]
            
            neighbors = self._find_k_nearest_neighbors(pd.DataFrame.copy(self.df), x)
            counts = self._get_counts(neighbors)
        
            membership = dict()
           #Esta seccion ejecuta la funcion (2) del paper en la cual se calcula la pertenencia
            for c in self.classes:
                try:
                    uci = 0.49 * (counts[c] / self.k)
                    if c == y:
                        uci += 0.51
                    membership[c] = uci
                except:
                    membership[c] = 0
                    
            memberships.append(membership)
        return memberships
        
        
    def _check_params(self, X, y):
        if type(self.k) != int:
            raise Exception('"k" should have type int')
        elif self.k >= len(y):
            raise Exception('"k" should be less than no of feature sets')
        #se comentaron las lineas para aceptar valores pares, como fue testeado en el paper
        #elif self.k % 2 == 0:
        #   raise Exception('"k" should be odd') 
            
        if type(self.plot) != bool:
            raise Exception('"plot" should have type bool')
            
def FKNN_eval(modelo, X_val, y_val):
  y_predict = modelo.predict(X_val)
  conf = confusion_matrix(y_val, y_predict)
  #Calculando las metricas a partir de la matriz de confusión
  TN = conf[0][0]
  FP = conf[0][1]
  TP = conf[1][1]
  FN = conf[1][0]
  #Calculando las métricas
  accuracy = accuracy_score(y_val, y_predict)
  recall = (TP)/(TP+FN)
  specificity = (TN)/(FP+TN)
  gmean = np.sqrt(recall*specificity)
  FPR = FP/(FP+TN)
  precision = TP/(TP+FP)
  fscore = 2*(precision*recall)/(precision+recall)

  #sns.heatmap(conf,annot=True, fmt="d", cmap="coolwarm");
  
  print('TP: ',TP,'TN: ',TN)
  print('FP: ',FP,'FN: ',FN)
  print('La accuracy para el modelo Fuzzy-KNN es: ', accuracy )
  print('La precision para el modelo Fuzzy-KNN es: ', precision )
  print('El recall para el modelo Fuzzy-KNN es: ', recall )
  print('La specificity para el modelo Fuzzy-KNN es: ', specificity)
  print('El G-mean para el modelo Fuzzy-KNN es: ', gmean)
  print('El FPR para el modelo Fuzzy-KNN es: ', FPR)
  print('El F1-Score para el modelo Fuzzy-KNN es: ', fscore)

  return [recall, specificity, gmean, FPR, precision, fscore]

# Tratamiento de la Data

Se repetira el experimento de Fuzzy-KNN con la data cruda, con la data normalizada con un min-max y con las exclusiones de las caracteristicas $X_3$ y $X_8$.

In [0]:
from sklearn.model_selection import train_test_split

#Partición de la data cruda
X = data_pulsar.drop(columns=["is_pulsar"])
y = data_pulsar["is_pulsar"]

X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.15, random_state=0) #random_state = 0, para mantener la particion
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.2, random_state=0)

#print(X.shape)
#print(X_trainval.shape)
print(X_train.shape)
#print(X_val.shape)


(12170, 8)


## Escalamiento de la data

In [0]:
#Escalamiento 
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(data_pulsar)
data_pulsar_scaled = pd.DataFrame(scaler.transform(data_pulsar), columns = head)
data_pulsar_scaled.describe()

#Partición de la data escalada
X_sca = data_pulsar_scaled.drop(columns=["is_pulsar"])
y_sca = data_pulsar_scaled["is_pulsar"]

X_trainval_sca, X_test_sca, y_trainval_sca, y_test_sca = train_test_split(X_sca, y_sca, test_size=0.15, random_state=0) #random_state = 0, para mantener la particion
X_train_sca, X_val_sca, y_train_sca, y_val_sca = train_test_split(X_trainval_sca, y_trainval_sca, test_size=0.2, random_state=0)


##Data sin la caracteristica $X_8$ *DM Skewness*

In [0]:
#Copiando data escala y eliminando la 8va caracteristica
X_train_8 = X_train_sca.copy()
X_train_8 = X_train_8.drop(columns=['skew_DM'])

X_val_8 =  X_val_sca.copy()
X_val_8 = X_val_8.drop(columns=['skew_DM'])

X_test_8 = X_test_sca.copy()
X_test_8 = X_test_8.drop(columns=['skew_DM'])

X_train_8.head()
print(X_train_8.shape)

(12170, 7)


## Data sin la caracteristica $X_8$  DM Skewness y $X_3$ IP Kurtosis

In [0]:
#Copiando data escala y eliminando la 3ra y 8va caracteristica
X_train_8y3 = X_train_sca.copy()
X_train_8y3 = X_train_8y3.drop(columns=['kurt_IP','skew_DM'])

X_val_8y3 = X_val_sca.copy()
X_val_8y3 = X_val_8y3.drop(columns=['kurt_IP','skew_DM'])

X_test_8y3 = X_test_sca.copy()
X_test_8y3 = X_test_8y3.drop(columns=['kurt_IP','skew_DM'])

#X_train_8y3.head()
print(X_train_8y3.shape)

(12170, 6)


# Entrenamiento y evaluación del modelo

##Utilizando Data Cruda
En el artículo se usa un factor $m=0.3$ y los valores de $k = [3,5,12,20]$, el código encontrado tuvo que ser modificado para que acepte un valor variable de $m$ y pueda aceptar valores de $k$ pares

In [0]:
k = [3,5,12,20]

In [0]:
for i in k:
  print("Modelo Fuzzy-KNN con k = ",i)
  FKNN = FuzzyKNN(k=i, m=0.3)
  FKNN.fit(X_train.values, y_train.tolist())
  #print("Modelo Fuzzy-KNN con k = ",i)
  FKNN_eval(FKNN, X_val.values, y_val)


Modelo Fuzzy-KNN con k =  3
TP:  220 TN:  2736
FP:  28 FN:  59
La accuracy para el modelo Fuzzy-KNN es:  0.9714097929674663
La precision para el modelo Fuzzy-KNN es:  0.8870967741935484
El recall para el modelo Fuzzy-KNN es:  0.7885304659498208
La specificity para el modelo Fuzzy-KNN es:  0.9898697539797395
El G-mean para el modelo Fuzzy-KNN es:  0.8834831398138159
El FPR para el modelo Fuzzy-KNN es:  0.010130246020260492
El F1-Score para el modelo Fuzzy-KNN es:  0.8349146110056925
Modelo Fuzzy-KNN con k =  5
TP:  219 TN:  2737
FP:  27 FN:  60
La accuracy para el modelo Fuzzy-KNN es:  0.9714097929674663
La precision para el modelo Fuzzy-KNN es:  0.8902439024390244
El recall para el modelo Fuzzy-KNN es:  0.7849462365591398
La specificity para el modelo Fuzzy-KNN es:  0.9902315484804631
El G-mean para el modelo Fuzzy-KNN es:  0.8816340098373411
El FPR para el modelo Fuzzy-KNN es:  0.009768451519536902
El F1-Score para el modelo Fuzzy-KNN es:  0.8342857142857142
Modelo Fuzzy-KNN con k =  

##Utilizando Data Escalada Min-Max

In [0]:
for i in k:
  print("Modelo Fuzzy-KNN con k = ",i)
  FKNN = FuzzyKNN(k=i, m=0.3)
  FKNN.fit(X_train_sca.values, y_train_sca.tolist())
  #print("Modelo Fuzzy-KNN con k = ",i)
  FKNN_eval(FKNN, X_val_sca.values, y_val_sca)

Modelo Fuzzy-KNN con k =  3
TP:  235 TN:  2744
FP:  20 FN:  44
La accuracy para el modelo Fuzzy-KNN es:  0.978968123562274
La precision para el modelo Fuzzy-KNN es:  0.9215686274509803
El recall para el modelo Fuzzy-KNN es:  0.8422939068100358
La specificity para el modelo Fuzzy-KNN es:  0.9927641099855282
El G-mean para el modelo Fuzzy-KNN es:  0.9144392602794887
El FPR para el modelo Fuzzy-KNN es:  0.00723589001447178
El F1-Score para el modelo Fuzzy-KNN es:  0.8801498127340824
Modelo Fuzzy-KNN con k =  5
TP:  235 TN:  2749
FP:  15 FN:  44
La accuracy para el modelo Fuzzy-KNN es:  0.9806112389089714
La precision para el modelo Fuzzy-KNN es:  0.94
El recall para el modelo Fuzzy-KNN es:  0.8422939068100358
La specificity para el modelo Fuzzy-KNN es:  0.9945730824891461
El G-mean para el modelo Fuzzy-KNN es:  0.9152720072513323
El FPR para el modelo Fuzzy-KNN es:  0.005426917510853835
El F1-Score para el modelo Fuzzy-KNN es:  0.888468809073724
Modelo Fuzzy-KNN con k =  12
TP:  231 TN:  

##Utilizando Data Escalada y sin la caractersitica $X_8$

In [0]:
for i in k:
  print("Modelo Fuzzy-KNN con k = ",i)
  FKNN = FuzzyKNN(k=i, m=0.3)
  FKNN.fit(X_train_8.values, y_train_sca.tolist())
  #print("Modelo Fuzzy-KNN con k = ",i)
  FKNN_eval(FKNN, X_val_8.values, y_val_sca)

Modelo Fuzzy-KNN con k =  3
TP:  233 TN:  2744
FP:  20 FN:  46
La accuracy para el modelo Fuzzy-KNN es:  0.9783108774235951
La precision para el modelo Fuzzy-KNN es:  0.9209486166007905
El recall para el modelo Fuzzy-KNN es:  0.8351254480286738
La specificity para el modelo Fuzzy-KNN es:  0.9927641099855282
El G-mean para el modelo Fuzzy-KNN es:  0.9105397147507911
El FPR para el modelo Fuzzy-KNN es:  0.00723589001447178
El F1-Score para el modelo Fuzzy-KNN es:  0.8759398496240601
Modelo Fuzzy-KNN con k =  5
TP:  235 TN:  2749
FP:  15 FN:  44
La accuracy para el modelo Fuzzy-KNN es:  0.9806112389089714
La precision para el modelo Fuzzy-KNN es:  0.94
El recall para el modelo Fuzzy-KNN es:  0.8422939068100358
La specificity para el modelo Fuzzy-KNN es:  0.9945730824891461
El G-mean para el modelo Fuzzy-KNN es:  0.9152720072513323
El FPR para el modelo Fuzzy-KNN es:  0.005426917510853835
El F1-Score para el modelo Fuzzy-KNN es:  0.888468809073724
Modelo Fuzzy-KNN con k =  12
TP:  231 TN: 

##Utilizando Data Escalada y sin la caractersitica $X_8$ y $X_3$

In [0]:
for i in k:
  print("Modelo Fuzzy-KNN con k = ",i)
  FKNN = FuzzyKNN(k=i, m=0.3)
  FKNN.fit(X_train_8y3.values, y_train_sca.tolist())
  #print("Modelo Fuzzy-KNN con k = ",i)
  FKNN_eval(FKNN, X_val_8y3.values, y_val_sca)

Modelo Fuzzy-KNN con k =  3
TP:  229 TN:  2740
FP:  24 FN:  50
La accuracy para el modelo Fuzzy-KNN es:  0.9756818928688794
La precision para el modelo Fuzzy-KNN es:  0.9051383399209486
El recall para el modelo Fuzzy-KNN es:  0.8207885304659498
La specificity para el modelo Fuzzy-KNN es:  0.9913169319826338
El G-mean para el modelo Fuzzy-KNN es:  0.9020319106484204
El FPR para el modelo Fuzzy-KNN es:  0.008683068017366137
El F1-Score para el modelo Fuzzy-KNN es:  0.8609022556390977
Modelo Fuzzy-KNN con k =  5
TP:  226 TN:  2743
FP:  21 FN:  53
La accuracy para el modelo Fuzzy-KNN es:  0.9756818928688794
La precision para el modelo Fuzzy-KNN es:  0.9149797570850202
El recall para el modelo Fuzzy-KNN es:  0.8100358422939068
La specificity para el modelo Fuzzy-KNN es:  0.9924023154848046
El G-mean para el modelo Fuzzy-KNN es:  0.8965943595172552
El FPR para el modelo Fuzzy-KNN es:  0.007597684515195369
El F1-Score para el modelo Fuzzy-KNN es:  0.8593155893536123
Modelo Fuzzy-KNN con k =  